In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading data and do EDA(Exploratory Data Analysis)

In [ ]:
import pandas as pd
import numpy as np
df_train = pd.read_csv('/kaggle/input/shopee-workshop-building-ann/HR-Employee-Attrition_train.csv')
df_train

<img src= 'https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F4342316%2F9c2161bc579fae87686285d953bdf56d%2Fdata%20description.png?generation=1591008771269892&alt=media'>

In [ ]:
#check if there is any missing value
df_train.info()

In [ ]:
#Check numerical data columns
print('There are %s numerical columns.'%(str(len(df_train.select_dtypes(include=['int64']).columns))))
df_train.select_dtypes(include=['int64']).columns

In [ ]:
#check categorical data columns 
print('There are %s numerical columns.'%(str(len(df_train.select_dtypes(include=['object']).columns))))
df_train.select_dtypes(include=['object']).columns

# Now we can divide all these 35 columns into 4 parts:
1. id column
2. target column
3. numerical columns
4. categorical columns

In [ ]:
id_col = 'EmployeeNumber'

target_col = 'Attrition'

#25 numerical columns
numerical_cols = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EnvironmentSatisfaction', 
                'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 
                'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 
                'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 
                'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

#8 categorical columns
categorical_cols = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 
                 'Over18', 'OverTime']

In [ ]:
#check 8 categorical columns
nunique_values = []
na_values = []
for col in categorical_cols:
    nunique_values.append(df_train[col].nunique())
    na_values.append(len(df_train.loc[df_train[col].isna()]))
df_train_stats_categorical = pd.DataFrame(list(zip(categorical_cols,nunique_values,na_values)),columns=['Column_Name','#Unique_Values','#Null_Value'])
df_train_stats_categorical

In [ ]:
#check Null columns
df_train_stats_categorical[df_train_stats_categorical['#Null_Value']>0]

In [ ]:
#check columns with only 1 value
df_train_stats_categorical[df_train_stats_categorical['#Unique_Values'] == 1]

The column 'Over18' only contains 1 unique value, therefore we could drop it.

In [ ]:
#check 25 numerical columns
na_values = []
for col in numerical_cols:
    na_values.append(len(df_train.loc[df_train[col].isna()]))
df_train_stats_numerical = pd.DataFrame(list(zip(numerical_cols,na_values)),columns=['Column_Name','#Null_Value'])
df_train_stats_numerical

In [ ]:
#check Null columns
df_train_stats_numerical[df_train_stats_numerical['#Null_Value']>0]

In [ ]:
df_train_stats_numerical = df_train[numerical_cols].describe().T
df_train_stats_numerical

In [ ]:
#check columns with only 1 value
df_train_stats_numerical[df_train_stats_numerical['std']==0]

Columns like 'EmployeeCount' and 'StandardHours' have 0 std, meaning all the values are the same, therefore we can drop them.

In [ ]:
#check categorical columns
def explore_categorical_columns(df,categorical_cols):
    nunique_values = []
    na_values = []
    for col in categorical_cols:
        nunique_values.append(df[col].nunique())
        na_values.append(len(df.loc[df[col].isna()]))
    df_stats_categorical = pd.DataFrame(list(zip(categorical_cols,nunique_values,na_values)),columns=['Column_Name','#Unique_Values','#Null_Value'])
    df_null = df_stats_categorical[df_stats_categorical['#Null_Value']>0]
    df_unique_value = df_stats_categorical[df_stats_categorical['#Unique_Values'] == 1]
    if len(df_null) > 0:
        print('Columns with Null value: %s'%(str(df_null['Column_Name'])))
    else:
        print('There is no Null values in the categorical columns')
    if len(df_unique_value) > 0:
        print('Columns with only 1 unique value: %s'%(str(list(df_unique_value['Column_Name']))))
    else:
        print('All categorical columns have more than 1 value.')
        
#check numerical columns
def explore_numerical_columns(df,numerical_cols):
    na_values = []
    for col in numerical_cols:
        na_values.append(len(df.loc[df[col].isna()]))
    df_stats_numerical = pd.DataFrame(list(zip(numerical_cols,na_values)),columns=['Column_Name','#Null_Value'])
    df_null = df_stats_numerical[df_stats_numerical['#Null_Value']>0]
    df_stats = df[numerical_cols].describe().T
    df_unique_value = df_stats[df_stats['std']==0]
    
    if len(df_null) > 0:
        print('Columns with Null value: %s'%(str(df_null['Column_Name'])))
    else:
        print('There is no Null values in the numerical columns')
    
    if len(df_unique_value) > 0:
        print('Columns with only 1 unique value: %s'%(str(list(df_unique_value.index))))
    else:
        print('All numerical columns have more than 1 value. \n')

#check categorical columns
print('Categorical_Columns'.center(50,"*"))
explore_categorical_columns(df_train,categorical_cols)
print('Numerical_Columns'.center(50,"*"))
#check numerical columns
explore_numerical_columns(df_train,numerical_cols)

- The column 'Over18' only contains 1 unique value, therefore we could drop it.
- Columns like 'EmployeeCount' and 'StandardHours' have 0 std, meaning all the values are the same, therefore we can drop them.

In [ ]:
# Check target variable
df_train[target_col].unique()

# Data Process

    # Scaling Numerical Data
<img src="https://miro.medium.com/max/1276/0*_apuT0HdrVYMUCh7">

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
def process_numerical_data(df,numerical_cols,drop_numerical_columns):
    sacalar = MinMaxScaler()
    # sacalar = StandardScaler() if using StandardScaler
    scale_numerical_cols = list(set(numerical_cols)-set(drop_numerical_columns))
    df_numerical = sacalar.fit_transform(df[scale_numerical_cols])
    df_numerical = pd.DataFrame(df_numerical,columns=scale_numerical_cols)
    return df_numerical

drop_numerical_columns = ['EmployeeCount', 'StandardHours']
df_train_numerical = process_numerical_data(df_train,numerical_cols,drop_numerical_columns)
df_train_numerical

    # Convert categorical columns into numerical columns
<img src="https://i.imgur.com/mtimFxh.png">
    

In [ ]:
df_train[categorical_cols]

In [ ]:
?pd.get_dummies

In [ ]:
def process_categorical_data(df,categorical_cols,drop_categorical_columns):
    df_categorical = pd.get_dummies(df[[var for var in categorical_cols if var not in drop_categorical_columns]])
    return df_categorical

In [ ]:
drop_categorical_columns = ['Over18']
df_train_categorical = process_categorical_data(df_train, categorical_cols, drop_categorical_columns)
df_train_categorical

In [ ]:
df_train_numerical.shape, df_train_categorical.shape

In [ ]:
df_train_features = pd.concat([df_train_numerical, df_train_categorical],axis=1)
df_train_features.shape

In [ ]:
df_train_features

In [ ]:
def process_label(df):
    target_col_dict = {'Yes': 1, 'No': 0}
    df_labels = df[target_col].map(target_col_dict).values
    return df_labels
df_train_labels = process_label(df_train)
df_train_labels.shape

In [ ]:
df_train_labels

In [ ]:
# split the data into train and test
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_train_features,df_train_labels,test_size=0.3,random_state=23)

# Set up and train the model

In [ ]:
from sklearn.neural_network import MLPClassifier
NN = MLPClassifier()
NN.fit(train_x,train_y)

In [ ]:
NN_predicted_y = NN.predict(test_x)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y,NN_predicted_y)

# Loading the test dataset and do the EDA

In [ ]:
df_test = pd.read_csv('/kaggle/input/shopee-workshop-building-ann/HR-Employee-Attrition_test.csv')
df_test

In [ ]:
df_test.info()

In [ ]:
print('Missing columns in test dataset(campared with train dataset): %s'%(str(set(df_train.columns)-set(df_test.columns))))

we can continue using the same 4 group column types:

In [ ]:
id_col = 'EmployeeNumber'

numerical_cols = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EnvironmentSatisfaction', 'HourlyRate', 
                  'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 
                  'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 
                  'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 
                  'YearsSinceLastPromotion', 'YearsWithCurrManager']

categorical_cols = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']

In [ ]:
#check 8 categorical columns
print('Categorical_Columns'.center(50,"*"))
explore_categorical_columns(df_test,categorical_cols)
print('Numerical_Columns'.center(50,"*"))
explore_numerical_columns(df_test,numerical_cols)

In [ ]:
drop_numerical_columns = ['EmployeeCount', 'StandardHours']
df_test_numerical = process_numerical_data(df_test,numerical_cols,drop_numerical_columns)
df_test_numerical

In [ ]:
# to check if there's any missing numerical columns in test dataset
set(df_train_numerical.columns)-set(df_test_numerical.columns)

In [ ]:
drop_categorical_columns = ['Over18']
df_test_categorical = process_categorical_data(df_test,categorical_cols,drop_categorical_columns)
df_test_categorical

In [ ]:
# to check if there's any missing categorical columns in test dataset
set(df_train_categorical.columns)-set(df_test_categorical.columns)

In [ ]:
df_test_features = pd.concat([df_test_numerical, df_test_categorical],axis=1)
df_test_features

# Make prediction on test dataset

In [ ]:
predicted_test = NN.predict(df_test_features)
predicted_test

In [ ]:
answer = df_test[['EmployeeNumber']]
answer['Attrition'] = ['Yes' if i == 1 else 'No' for i in predicted_test]
answer

In [ ]:
answer.to_csv('submission.csv',index=False)

In [ ]:
#You can use below code to download the answer:

from IPython.display import FileLink
FileLink(r'submission.csv')

# GridSearch to find the optimal parameter setting
Most used parameter

1）The ith element represents the number of neurons in the ith hidden layer.
    example: ``hidden_layer_sizes``=(10)
    
2）activation function
    example: ``activation``="relu"

3）The solver for weight optimization. {'lbfgs', 'sgd', 'adam'}
    example: ``solver``='adam'

4）L2 penalty (regularization term) parameter
    example: ``alpha``=0.0001

5）Size of minibatches for stochastic optimizers.
    If the solver is 'lbfgs', the classifier will not use minibatch.
    When set to 'auto', 'batch_size=min(200, n_samples)'
    example: ``batch_size``='auto'

6）Learning rate schedule for weight updates.
    example: ``learning_rate``="constant"

7）The initial learning rate used. It controls the step-size in updating the weights. Only used when solver='sgd' or 'adam'.
    example: ``learning_rate_init``=0.001

8）Maximum number of iterations. The solver iterates until convergence
    example: ``max_iter``=200

9）Tolerance for the optimization. When the loss or score is not improving
    by at least ``tol`` for ``n_iter_no_change`` consecutive iterations,
    unless ``learning_rate`` is set to 'adaptive', convergence is
    considered to be reached and training stops.
    example: ``tol``=1e-4

In [ ]:
%%time
import time
start_time = time.time()
NN_new = MLPClassifier(max_iter=1000)
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (50,100,50,25)],
    'activation': ['tanh', 'relu','logistic'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.001, 0.01],
    'learning_rate': ['constant','adaptive'],
}
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(NN_new, parameter_space, n_jobs=-1, cv=10)
clf.fit(train_x, train_y)
print('Time taken for training the model: '+ str(time.time() - start_time))

In [ ]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

In [ ]:
# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
y_true, y_pred = test_y , clf.predict(test_x)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
test_y_pred = clf.predict(df_test_features)

In [ ]:
df_result = pd.DataFrame.from_dict(dict({'EmployeeNumber':list(df_test['EmployeeNumber']),
                                         'Attrition':['Yes' if i == 1 else 'No' for i in test_y_pred]}))
df_result

In [ ]:
temp = pd.merge(df_result,answer,on='EmployeeNumber',how='inner')
temp.loc[~(temp['Attrition_x']==temp['Attrition_y'])]